# 8. Logistička regresija

In [1]:
import random
import math
import os
from test_v08 import *

random.seed(0)

SENT_FILENAME = "sentiment_lexicon.csv"
DATA_PATH = "./data/"

### 8.1 Leksikon sentimenta

Funkcija **read_sentiments()** ima sljedeće ulazne i izlazne parametre
* ulaz:
    * **filename** putanja datoteke sentimenata
* izlaz:
    * rječnik sentimenta koji za ključ ima riječ, a za vrijednost 1 ako je riječ pozitivna, odnosno 0 ako je negativna.
    


In [2]:
def read_sentiment(filename):
    sentiments = {}
    with open(filename) as fp:
        for line in fp:
            w, k = line.rstrip().split(",")
            sentiments[w.lower()] = 1 if k == "+" else 0
    return sentiments

sentiments = read_sentiment(SENT_FILENAME)

### 8.2 Skup podataka

Funkcija **read_datasets()** ima sljedeće ulazne i izlazne parametre:
* ulaz:
    * **path** putanja do dokumenata
* izlaz:
    * lista svih 
    * **testset** - skup podataka za testiranje

In [3]:
def read_dataset(data_path):
    klasses = [klass for klass in os.listdir(data_path) if not klass.startswith('.')]
    dataset = []
    for klass in klasses:
        for filename in os.listdir(data_path + klass + "/"):
            if not filename.endswith(".txt"):
                continue
            filename = data_path + klass + "/" + filename
            dataset.append((filename, klass))
    return dataset

dataset = read_dataset("./data/")

In [4]:
def make_features1(filename, sentiments):
    with open(filename) as fp:
        words = fp.read().strip().split()
        num_pos = 0
        num_neg = 0
        num_negation = 0
        num_pronouns = 0
        num_exclamations = 0
        num_words = 0
        for w in words:
            if w in sentiments:
                if sentiments[w] == 1:
                    num_pos += 1
                else:
                    num_neg += 1

        return [num_pos,
                num_neg]

    
def make_features2(filename, sentiments):
    with open(filename) as fp:
        words = fp.read().strip().split()
        num_pos = 0
        num_neg = 0
        num_negation = 0
        num_pronouns = 0
        num_exclamations = 0
        num_words = 0
        for w in words:
            if w in sentiments:
                if sentiments[w] == 1:
                    num_pos += 1
                else:
                    num_neg += 1
            elif w in ("no", "not", "none") or w.endswith("n't"):
                num_negation += 1
            elif w in ("i", "you", "me"):
                num_pronouns += 1
            elif w in ("!", "?"):
                num_exclamations += 1
            num_words += 1

        return [num_pos,
                num_neg,
                num_negation,
                num_pronouns,
                num_exclamations,
                math.log(num_words)]
    

make_features1(dataset[0][0], sentiments)


[24, 16]

In [5]:
def transform_dataset(dataset, features_function, sentiments):
    return [(features_function(filename, sentiments), 1 if klass=="pos" else 0)  for filename, klass in dataset]

featureset = transform_dataset(dataset, make_features1, sentiments)
featureset

[([24, 16], 0),
 ([32, 32], 0),
 ([24, 30], 0),
 ([22, 18], 0),
 ([21, 25], 0),
 ([7, 20], 0),
 ([25, 18], 0),
 ([25, 28], 0),
 ([62, 46], 0),
 ([21, 20], 0),
 ([49, 49], 0),
 ([46, 52], 0),
 ([15, 14], 0),
 ([28, 15], 0),
 ([29, 43], 0),
 ([23, 42], 0),
 ([20, 15], 0),
 ([59, 49], 0),
 ([9, 19], 0),
 ([32, 35], 0),
 ([30, 28], 0),
 ([19, 26], 0),
 ([41, 29], 0),
 ([36, 33], 0),
 ([25, 30], 0),
 ([17, 5], 0),
 ([50, 55], 0),
 ([24, 25], 0),
 ([31, 11], 0),
 ([19, 16], 0),
 ([30, 44], 0),
 ([25, 22], 0),
 ([34, 30], 0),
 ([26, 32], 0),
 ([51, 61], 0),
 ([25, 44], 0),
 ([18, 20], 0),
 ([38, 32], 0),
 ([29, 24], 0),
 ([22, 8], 0),
 ([17, 10], 0),
 ([24, 33], 0),
 ([72, 70], 0),
 ([23, 34], 0),
 ([14, 15], 0),
 ([15, 11], 0),
 ([44, 32], 0),
 ([44, 30], 0),
 ([34, 40], 0),
 ([20, 27], 0),
 ([28, 41], 0),
 ([49, 50], 0),
 ([27, 37], 0),
 ([23, 11], 0),
 ([36, 41], 0),
 ([27, 14], 0),
 ([14, 23], 0),
 ([33, 28], 0),
 ([5, 16], 0),
 ([36, 49], 0),
 ([16, 19], 0),
 ([21, 25], 0),
 ([22, 23], 0

In [6]:
def split_featureset(dataset, train_percent=0.8):
    random.shuffle(dataset)
    train_size = round(len(dataset) * train_percent)
    return dataset[:train_size], dataset[train_size:] 

trainset, testset = split_featureset(featureset, 0.8)

trainset[:10], testset[:10]

([([32, 41], 1),
  ([22, 32], 1),
  ([41, 29], 0),
  ([42, 38], 1),
  ([27, 24], 1),
  ([22, 32], 0),
  ([20, 25], 0),
  ([32, 17], 1),
  ([5, 16], 0),
  ([34, 47], 0)],
 [([18, 25], 1),
  ([23, 13], 1),
  ([44, 38], 1),
  ([29, 22], 1),
  ([27, 37], 0),
  ([41, 21], 1),
  ([22, 23], 1),
  ([22, 23], 0),
  ([31, 30], 1),
  ([28, 33], 0)])

In [7]:
def dot(x, y):
    return sum(xi * yi for xi, yi in zip(x, y))


def sigmoid(z):
    return 1 / (1 + math.exp(-z))

In [14]:
def training(trainset, num_iters=500, learn_rate=0.01, batch_size=1024):
    x, y = trainset[0]
    m = len(trainset)
    n = len(x)  # velicina vektora

    # inicijalizacija težina
    w = [0 for _ in range(n)]
    b = 0
    for i in range(num_iters):
        loss = 0
        sample = random.sample(trainset, batch_size)
        for x, y in sample:
            yh = sigmoid(dot(w, x) + b)
            loss += -(y * math.log(yh) + (1 - y) * math.log(1 - yh))

            dw = [((yh - y) * xi) for xi in x]
            db = (yh - y)

            w = [wi - learn_rate * dwi for wi, dwi in zip(w, dw)]
            b = b - learn_rate * db

        loss /= batch_size
        if (i + 1) % 100 == 0:
            print(f"#{i} Loss: {loss}")

    return w, b


def predict(w, b, x):
    if sigmoid(dot(w, x) + b) > 0.5:
        return 1
    return 0


def testing(testset, w, b):
    tp = fp = fn = tn = 0
    for x, y in testset:
        yh = predict(w, b, x)
        if yh == 1:
            if y == 1:
                tp += 1
            else:
                fp += 1
        else:
            if y == 1:
                fn += 1
            else:
                tn += 1
    acc = (tp + tn) / (tp + fp + fn + tn)
    p = tp / (tp + fp)
    r = tp / (tp + fn)
    f1 = 2 * p * r / (p + r)
    print(tp, fp, fn, tn)
    return acc, p, r, f1

In [13]:
for _ in range(10):
    trainset, testset = split_featureset(featureset, 0.8)

    w, b = training(trainset, num_iters=2000, learn_rate=0.0001, batch_size=len(trainset))
    acc, p, r, f1 = testing(testset, w, b)
    print(f"acc={acc:.4f} p={p:.4f} r={r:.4f} f1={f1:.4f}")

#99 Loss: 0.671354713406062
#199 Loss: 0.655871404650063
#299 Loss: 0.6447511539815207
#399 Loss: 0.6366224292449288
#499 Loss: 0.6305820846475563
#599 Loss: 0.6260283676846371
#699 Loss: 0.6225507096853554
#799 Loss: 0.6198666389265121
#899 Loss: 0.6177760456240464
#999 Loss: 0.616135634112632
#1099 Loss: 0.6148356681134817
#1199 Loss: 0.6138046418545079
#1299 Loss: 0.6129782546056073
#1399 Loss: 0.6123138960207706
#1499 Loss: 0.6117775339820457
#1599 Loss: 0.6113429500460642
#1699 Loss: 0.6109883857882608
#1799 Loss: 0.6107009791201456
#1899 Loss: 0.6104655244913305
#1999 Loss: 0.6102719674471152
15 6 6 13
acc=0.7000 p=0.7143 r=0.7143 f1=0.7143
#99 Loss: 0.6684760120656523
#199 Loss: 0.6512557425315652
#299 Loss: 0.6386718855577607
#399 Loss: 0.6293057003752163
#499 Loss: 0.6222119393879113
#599 Loss: 0.6167589916945311
#699 Loss: 0.612511814509015
#799 Loss: 0.6091657712433081
#899 Loss: 0.6065071363741321
#999 Loss: 0.6043740153986494
#1099 Loss: 0.602650200689715
#1199 Loss: 0.601

### 8.2. Osobine

Definirat ćemo 3 funkcije osobina koje pretvaraju podatak koji se klasificira $d$ u vektor realnih brojeva $x = \lbrack x_1, x_2, ..., x_n \rbrack$.

Funkcija **features_len()** ima sljedeće ulazne i izlazne podatke:
* ulaz:
    * **d** - ime
* izlaz:
    * vektor oblika $\lbrack x_1 \rbrack$ gdje je $x_1$ broj znakova u imenu $d$

In [4]:
def features_len(d):
    return [len(d)]

_ = test(features_len, ("issac", ), [5])
_ = test(features_len, ("ada", ), [3])

OK	features_len('issac',) 
	=> [5]
	== [5]

OK	features_len('ada',) 
	=> [3]
	== [3]



In [5]:
LETTERS = "abcdefghijklmnopqrstuvwxyz"

LETTERS

'abcdefghijklmnopqrstuvwxyz'

Funkcija **features_1()** ima sljedeće ulazne i izlazne podatke:
* ulaz:
    * **d** - ime
* izlaz:
    * vektor oblika $\lbrack x_a, x_b, ..., x_z] \rbrack$ gdje je $x_a = 1$ ako je zadnje slovo imena $d$ jednako "a", $x_b = 1$ ako je zadnje slovo imena $d$ jednako "b", itd. U suprotnom su jednaki $0$.
    
Napomena: koristiti konstantu `LETTERS` koja sadrži sva slova engleske abecede.

In [6]:
def features_1(doc):
    vector = [0] * len(LETTERS)
    vector[LETTERS.index(doc[-1])] = 1
    return vector
    
_ = test(features_1, ("issac", ), [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
_ = test(features_1, ("ada", ), [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

OK	features_1('issac',) 
	=> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, '...']
	== [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, '...']

OK	features_1('ada',) 
	=> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, '...']
	== [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, '...']



In [7]:
TWOLETTERS = [l1 + l2 for l1 in LETTERS for l2 in LETTERS]
TWOLETTERS

['aa',
 'ab',
 'ac',
 'ad',
 'ae',
 'af',
 'ag',
 'ah',
 'ai',
 'aj',
 'ak',
 'al',
 'am',
 'an',
 'ao',
 'ap',
 'aq',
 'ar',
 'as',
 'at',
 'au',
 'av',
 'aw',
 'ax',
 'ay',
 'az',
 'ba',
 'bb',
 'bc',
 'bd',
 'be',
 'bf',
 'bg',
 'bh',
 'bi',
 'bj',
 'bk',
 'bl',
 'bm',
 'bn',
 'bo',
 'bp',
 'bq',
 'br',
 'bs',
 'bt',
 'bu',
 'bv',
 'bw',
 'bx',
 'by',
 'bz',
 'ca',
 'cb',
 'cc',
 'cd',
 'ce',
 'cf',
 'cg',
 'ch',
 'ci',
 'cj',
 'ck',
 'cl',
 'cm',
 'cn',
 'co',
 'cp',
 'cq',
 'cr',
 'cs',
 'ct',
 'cu',
 'cv',
 'cw',
 'cx',
 'cy',
 'cz',
 'da',
 'db',
 'dc',
 'dd',
 'de',
 'df',
 'dg',
 'dh',
 'di',
 'dj',
 'dk',
 'dl',
 'dm',
 'dn',
 'do',
 'dp',
 'dq',
 'dr',
 'ds',
 'dt',
 'du',
 'dv',
 'dw',
 'dx',
 'dy',
 'dz',
 'ea',
 'eb',
 'ec',
 'ed',
 'ee',
 'ef',
 'eg',
 'eh',
 'ei',
 'ej',
 'ek',
 'el',
 'em',
 'en',
 'eo',
 'ep',
 'eq',
 'er',
 'es',
 'et',
 'eu',
 'ev',
 'ew',
 'ex',
 'ey',
 'ez',
 'fa',
 'fb',
 'fc',
 'fd',
 'fe',
 'ff',
 'fg',
 'fh',
 'fi',
 'fj',
 'fk',
 'fl',
 'fm',

Funkcija **features_2()** ima sljedeće ulazne i izlazne podatke:
* ulaz:
    * **d** - ime
* izlaz:
    * vektor oblika $\lbrack x_{aa}, x_{ab}, ..., x_{az}, x_{ba}, x_{bb}, ..., x_{bz}, ..., x_{za}, x_{zb}, ..., x_{zz} \rbrack$ gdje je $x_{aa} = 1$ ako su zadnja dva slova imena $d$ jednaka "aa", $x_{ab} = 1$ ako je zadnje slovo imena $d$ jednaka "ab", itd. U suprotnom su jednaki $0$.
    
Napomena: koristiti konstantu `TWOLETTERS` koja sadrži sve moguće parove od dva slova engleske abecede.

In [8]:
def features_2(doc):
    vector = [0] * len(TWOLETTERS)
    vector[TWOLETTERS.index(doc[-2:])] = 1
    return vector

_ = test(features_2, ("issac", ), [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
_ = test(features_2, ("ada", ), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

OK	features_2('issac',) 
	=> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, '...']
	== [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, '...']

OK	features_2('ada',) 
	=> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '...']
	== [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '...']



### 8.3. Predviđanje

Funkcija **predict()** ima sljedeće ulazne i izlazne podatke:
* ulaz:
    * **W** - vektor težina oblika $W=\lbrack w_1, w_2, ..., w_n, b \rbrack$ (zadnji element je pristranost $b$.
    * **d** - ime
    * **features** - funkcija koja ime $d$ pretvara u vektor osobina $\lbrack x_1, x_2, ..., x_n \rbrack$
* izlaz:
    * $y=1$ za "female", odnosno $y=0" za male
    
Predviđanje se vrši na sljedeći način:
1. dokument $d$ se pretvori u vektor osobina $X = \lbrack x_1, x_2, ..., x_n, 1 \rbrack$ (zadnji element je $1$ koji će se množiti s pristranošću $b$
2. izračuna se $y = \frac{1}{1 + e^{-W \cdot X}}$ gdje je skalarni produkt $W \cdot X = \sum_{i=1}^{n} {w_i x_i} + b$
3. Ako je $y \gt 0.5$ onda vrati 1 ("female"), u suprotnom 0 ("male")

In [9]:
def predict(W, d, features):
    X = features(d) + [1]
    y = 1 / (1 + math.exp(-sum(w * x for w, x in zip(W, X))))
    return 1 if y > 0.5 else 0



for features, ys in zip([features_len, features_1, features_2], [(1, 1), (0, 0), (0, 0)]):
    random.seed(0)
    for y in ys:
        W = [random.randrange(-10, 10) for _ in range(len(features("ada")))]
        test(predict, (W, "issac", features), y)


OK	predict([2], 'issac', <function features_len at 0x7fb1a9e399e0>) 
	=> 1
	== 1

OK	predict([3], 'issac', <function features_len at 0x7fb1a9e399e0>) 
	=> 1
	== 1

OK	predict([2, 3, -9, -2, 6, 5, 2, -1, 5, 1, '...'], 'issac', <function features_1 at 0x7f... 
	=> 0
	== 0

OK	predict([5, 7, -7, 1, 3, 0, 9, -4, 7, 5, '...'], 'issac', <function features_1 at 0x7fb... 
	=> 0
	== 0

OK	predict([2, 3, -9, -2, 6, 5, 2, -1, 5, 1, '...'], 'issac', <function features_2 at 0x7f... 
	=> 0
	== 0

OK	predict([-1, 1, -8, -3, 4, 1, 6, -9, 2, 3, '...'], 'issac', <function features_2 at 0x7... 
	=> 0
	== 0



### 8.4. Učenje

Funkcija **train()** ima sljedeće ulazne i izlazne podatke:
* ulaz:
    * **trainset** - skup podataka za treniranje oblika $\lbrack (d, c) \rbrack$.
    * **features** - funkcija koja ime $d$ pretvara u vektor osobina $\lbrack x_1, x_2, ..., x_n \rbrack$
    * **epoch** - broj epoha $epoch$
    * **rate** - rata učenja $\mu$
* izlaz:
    * model $W$ oblika $\lbrack w_1, w_2, ..., w_n, b \rbrack$
    
Učenje (treniranje) se vrši na sljedeći način:
1. Pretvoriti trainset $\lbrack (d, c) \rbrack$ u featureset $\lbrack (\lbrack x_1, ..., x_n, 1 \rbrack, y) \rbrack$
    * $d$ se pretvara u $X = \lbrack x_1, ..., x_n, 1 \rbrack$ korištenjem funkcije osobina `features()`
    * $c$ se pretvara u $y$ ($y=1$ ako je $c$ "female", odnosno $y=0$ ako je $c$ "male")
2. Stvoriti vektor težina $W = \lbrack w_1, ..., w_n, b \rbrack$
3. za svaku epohu:
    1. promiješati featureset (koristiti random.shuffle)
    2. za svaki par $(X, y)$ iz featureseta
        1. izračunati $\hat{y} = \frac{1}{1 + e^{W \cdot X}}$ gdje je skalarni produkt $W \cdot X = \sum_{i=1}^{n} {w_i x_i} + b$
        2. izračunati gradijent $\nabla L = \begin{bmatrix} \frac{\partial L}{\partial w_1} \\ \vdots \\ \frac{\partial L}{\partial w_n} \\ \frac{\partial L}{\partial b} \end{bmatrix}$ gdje je $\frac{\partial L}{\partial w_j} = (\hat{y} - y) * x_j$ za $j=1,...,n$ i $\frac{\partial L}{\partial b} = (\hat{y} - y)$
        3. izračunati nove težine $W = W - \mu \nabla L$
4. vratiti $W$

Napomena: staviti print(epoch) prije koraka A.

In [10]:
def train(trainset, features, epochs=15, rate=0.1):
    featureset = [(features(d) + [1], 1 if c == "female" else 0) for d, c in trainset]  # [1] is bias
    size = len(featureset[0][0])
    W = [0] * size
    S = 0
    for epoch in range(epochs):
        cost = 0
        random.shuffle(featureset)
        for X, y in featureset:
            Y = [y, 1 - y]

            yh = 1 / (1 + math.exp(-sum(w * x for w, x in zip(W, X))))
            Yh = [yh, 1 - yh]
            
            l = - (y * math.log(yh) + (1 - y) * math.log(1 - yh))
            cost += l

            #error += (yh - y) ** 2 / len(X)
            # DL_Dw = [sum((yh - y) for yh, y in zip(Yh, Y)) * x for x in X]
            DL_Dw = [(yh - y) * x for x in X]
            
            S += sum([dl_dw for w, dl_dw in zip(W, DL_Dw)])
            

            W = [w - rate * dl_dw for w, dl_dw in zip(W, DL_Dw)]
        cost /= len(featureset)
        print("{}. {:.4f}".format(epoch, cost, S/len(featureset)))
        
    
    return W

random.seed(0)
W = test(train, (trainset, features_len, 20, 0.1), [0.03140851227831187, -0.19269386591616966])
W

0. 1.0717
1. 1.0769
2. 1.0820
3. 1.0753
4. 1.0872
5. 1.0756
6. 1.0841
7. 1.0825
8. 1.0927
9. 1.0814
10. 1.0915
11. 1.0779
12. 1.0913
13. 1.0718
14. 1.0961
15. 1.0952
16. 1.0995
17. 1.0749
18. 1.0801
19. 1.0900
OK	train([('danette', 'female'), ('leia', 'female'), ('walker', 'male'), ('friedrich', '... 
	=> [0.03140851227831187, -0.19269386591616966]
	== [0.03140851227831187, -0.19269386591616966]



[0.03140851227831187, -0.19269386591616966]

### 8.5. Tablica slučaja

Funkcija **contingency()** ima sljedeće ulazne i izlazne podatke:
* ulaz:
    * **W** - vektor težina oblika $W=\lbrack w_1, w_2, ..., w_n, b \rbrack$ (zadnji element je pristranost $b$.
    * **testset** - testni skup podataka
    * **classes** - lista klasa oblika ["male", female"]
    * **features** - funkcija koja ime $d$ pretvara u vektor osobina $\lbrack x_1, x_2, ..., x_n \rbrack$
* izlaz:
    * **tp** - True Positives
    * **fp** - False Positives
    * **fn** - False Negatives
    * **tn** - True Negatives

Prilikom izgradnje tablice slučaja, za svaki $(d, c)$ iz skupa za testiranje se, pomoću funkcije `predict()`, predvidi klasa $\hat{c}$. 
Usporedbom $c$ i $\hat{c}$ se popune vrijednost tablice:

|            | female (1) | male (0) |
|------------|------------|----------|
| female (1) | $tp$       | $fp$     |
| male (0)   | $fn$       | $tn$     |

gdje je:
* $tp$ - True Positive ako $c$ = "female" i $\hat{c}$ = "female"
* $fp$ - False Positive ako $c$ = "male" i $\hat{c}$ = "female"
* $tn$ - True Negative ako $c$ = "female" i $\hat{c}$ = "male"
* $fn$ - False Negative ako $c$ = "male" i $\hat{c}$ = "male"


* izlaz:
    * **accuracy** - $A$ točnost
    * **precission** - $P$ preciznost
    * **recall** - $R$ odziv
    * **f1** - $F1$ mjera

Prilikom testiranja, za svaki $(d, c)$ iz skupa za testiranje se odredi pomoću funkcije `predict()` predvidi klasa $\hat{c}$. 
Usporedbom $c$ i $\hat{c}$ se popune vrijednost matrice:

|            | female (1) | male (0) |
|------------|------------|----------|
| female (1) | $tp$       | $fp$     |
| male (0)   | $fn$       | $tn$     |

gdje je:
* $tp$ - True Positive ako $c$ = "female" i $\hat{c}$ = "female"
* $fp$ - False Positive ako $c$ = "male" i $\hat{c}$ = "female"
* $tn$ - True Negative ako $c$ = "female" i $\hat{c}$ = "male"
* $fn$ - False Negative ako $c$ = "male" i $\hat{c}$ = "male"

Zatim se izračuna: 
* $A = \frac{tp + fn}{tp + fp + tn + fn}$
* $P = \frac{tp}{tp + fp}$
* $R = \frac{tp}{tp + fn}$
* $F1 = 2 \frac{PR}{P+R}$



In [11]:
def contingency(W, testset, features):
    evalset = [(predict(W, d, features), 1 if c == "female" else 0) for d, c in testset]
    tp, fp, fn, tn = 0, 0, 0, 0
    for ch, c in evalset:
        if ch == 1 == c:
            tp += 1
        elif ch == 1 != c:
            fp += 1
        elif ch == 0 != c:        
            fn += 1
        elif ch == 0 == c:                          
            tn += 1
    return tp, fp, fn, tn

_ = test(contingency, (W, testset, features_len), (389, 200, 602, 398))

OK	contingency([0.03140851227831187, -0.19269386591616966], [('bret', 'male'), ('marcie', 'fem... 
	=> (389, 200, 602, 398)
	== (389, 200, 602, 398)



### 8.6. Evaluacija

Funkcija **evaluate()** ima sljedeće ulazne i izlazne podatke:
* ulaz:
    * **W** - vektor težina oblika $W=\lbrack w_1, w_2, ..., w_n, b \rbrack$ (zadnji element je pristranost $b$.
    * **testset** - testni skup podataka
    * **classes** - lista klasa oblika ["male", female"]
    * **features** - funkcija koja ime $d$ pretvara u vektor osobina $\lbrack x_1, x_2, ..., x_n \rbrack$
* izlaz:
    * **accuracy** - $Acc$ točnost
    * **precission** - $P$ preciznost
    * **recall** - $R$ odziv
    * **f1** - $F1$ mjera

Prilikom evaluacije, koristiti **contingency()** za određivanje TP, FP, FN i TN, a zatim izračunati
* $Acc = \frac{tp + fn}{tp + fp + tn + fn}$
* $P = \frac{tp}{tp + fp}$
* $R = \frac{tp}{tp + fn}$
* $F1 = 2 \frac{PR}{P+R}$



In [12]:
def evaluate(W, testset, features):
    tp, fp, fn, tn = contingency(W, testset, features)
            
    a = (tp + tn) / (tp + fp + tn + fn)
    p = tp / (tp + fp)
    r = tp / (tp + fn)
    f1 = 2 * p * r / (p + r)
    
    return a, p, r, f1

_ = test(evaluate, (W, testset, features_len), (0.49528005034612965, 0.6604414261460102, 0.39253279515640765, 0.49240506329113926))

OK	evaluate([0.03140851227831187, -0.19269386591616966], [('bret', 'male'), ('marcie', 'fem... 
	=> (0.49528005034612965, 0.6604414261460102, 0.39253279515640765, 0.49240506329113926)
	== (0.49528005034612965, 0.6604414261460102, 0.39253279515640765, 0.49240506329113926)



## 8.7. Učenje i testiranje za osobine "features_1"

In [13]:
features = features_1
print("Učenje")

W = train(trainset, features, epochs=20, rate=0.1)
print("W =", W)
print()
print("Testiranje")
tp, fp, fn, tn = contingency(W, testset, features)
acc, p, r, f1 = evaluate(W, testset, features)
print("TP: {}\nFP: {}\nFN: {}\nTN: {}".format(tp, fp, fn, tn))
print()
print("Acc: {:.4}\nP: {:.4}\nR: {:.4}\nF1: {:.4}".format(acc, p, r, f1))

Učenje
0. 0.4922
1. 0.4780
2. 0.4758
3. 0.4762
4. 0.4767
5. 0.4760
6. 0.4764
7. 0.4764
8. 0.4754
9. 0.4758
10. 0.4750
11. 0.4741
12. 0.4752
13. 0.4746
14. 0.4752
15. 0.4755
16. 0.4745
17. 0.4764
18. 0.4748
19. 0.4748
W = [5.115821995670993, 0.011058575198413467, -2.7920115265088565, -0.6063846077238806, 2.0492808593272542, -1.9282120614312228, -0.514648361511249, 1.1426243658731368, 2.60483803343152, 0.12399046048637327, -2.43684096909896, 0.9905941110808729, -0.9532009716457673, 0.9843639616395721, -0.7625453859289402, -1.738650245090031, 0.0, -0.49169395303012403, -0.003887023562025589, -0.041609900715398745, -0.31408603077403974, -2.478876703951599, -0.3160371896254602, 0.7659049447574175, 1.3769686296585344, -0.6169340548292286, -0.8301730483026415]

Testiranje
TP: 924
FP: 336
FN: 67
TN: 262

Acc: 0.7464
P: 0.7333
R: 0.9324
F1: 0.821


# 8.8. Učenje i testiranje za osobine "features_2"

In [14]:
features = features_2
print("Učenje")

W = train(trainset, features, epochs=20, rate=0.1)
print("W =", W)
print()
print("Testiranje")
tp, fp, fn, tn = contingency(W, testset, features)
acc, p, r, f1 = evaluate(W, testset, features)
print("TP: {}\nFP: {}\nFN: {}\nTN: {}".format(tp, fp, fn, tn))
print()
print("Acc: {:.4}\nP: {:.4}\nR: {:.4}\nF1: {:.4}".format(acc, p, r, f1))

Učenje
0. 0.5296
1. 0.4638
2. 0.4458
3. 0.4379
4. 0.4329
5. 0.4289
6. 0.4260
7. 0.4243
8. 0.4235
9. 0.4209
10. 0.4202
11. 0.4194
12. 0.4181
13. 0.4164
14. 0.4170
15. 0.4163
16. 0.4162
17. 0.4150
18. 0.4146
19. 0.4146
W = [-0.7273185582535121, -0.42498932271467005, -1.2264639245688993, -0.5690227096538764, 1.3888937939725026, 0.12393095292408296, 0.20196547066970846, 0.9431878483097802, -0.7491582427608195, -0.728324313017376, -1.596980051605257, 0.5583804595833325, -1.1540417548076034, -0.3724314217194451, -0.7727808227181182, 0.0, 0.0, -1.3313767972794925, -1.6228288324522655, -0.14482168366683656, -0.35744157668386417, -1.6162018757523857, -1.588366601334044, -0.7562232106149661, 0.07881044338397891, -0.7073368682403373, 2.9583635133716015, -1.5881371956904733, 0.0, 0.0, 0.600040163833747, 0.0, 0.0, 0.0, 2.0729457270225624, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1090915354541562, 0.0, 0.0, 0.0, 0.8454682486171686, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24698040787447417, 0.0, 4.226926678161243, 0.0, 0.0, 0.

In [15]:
def make_tfidf_features(dataset):
    v = set()
    for d, _ in dataset:
        v.update(d)
        
    v = sorted(v)
    tf = {}
    idf = {w: 0 for w in v}
    
    for d, _ in dataset:
        tf[d] = [0] * len(v)
        for w in d:
            tf[d][v.index(w)] += 1
            idf[w] += 1
        n = len(d)
        tf[d] = [f / n for f in tf[d]]

    N = len(tf)
    idf = {w: math.log(N / f) for w, f in idf.items()}

    tfidf = {d: [f * idf[w] for f, w in zip(tf[d], v)] for d, _ in dataset}
    
    return tfidf
    

    
tfidf = make_tfidf_features(trainset + testset)

def features_tfidf(d):
    return tfidf[d]


features = features_tfidf
W = train(trainset, features_tfidf, epochs=15, rate=0.1)
evaluate(W, testset, features)

0. 0.6243
1. 0.5949
2. 0.5869
3. 0.5836
4. 0.5793
5. 0.5754
6. 0.5737
7. 0.5727
8. 0.5707
9. 0.5695
10. 0.5683
11. 0.5676
12. 0.5671
13. 0.5660
14. 0.5667


(0.7086217747010699,
 0.7075471698113207,
 0.9081735620585267,
 0.7954043305346885)